In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.utils import shuffle
import random

PDBs = pickle.load(open('Datasets/PDBs_RDKit_BFE.pkl', 'rb'))
df = pd.read_csv('Datasets/Final_data_DDG.csv')

In [3]:
df

,Unnamed: 0,Ids,pb_host_VDWAALS,pb_guest_VDWAALS,pb_complex_VDWAALS,gb_host_1-4EEL,gb_guest_1-4EEL,gb_Complex_1-4EEL,gb_host_EELEC,gb_guest_EELEC,gb_Complex_EELEC,gb_host_EGB,gb_guest_EGB,gb_Complex_EGB,gb_host_ESURF,gb_guest_ESURF,gb_Complex_ESURF,Ex _G_(kcal/mol)
0,0,cd-set1--acd-guest-1,-28.5717,-0.0564,-40.2895,423.5761,12.2487,435.8248,-428.9520,16.4764,-402.7811,-96.4471,-67.4372,-162.2809,5.1691,1.4344,4.9718,1.575
1,1,cd-set1--acd-guest-2,-27.1334,-0.3831,-44.0766,412.6663,16.3135,428.9798,-397.9784,11.5637,-376.0288,-106.4154,-66.7869,-176.4630,5.0726,1.7438,4.9888,3.533
2,2,cd-set1--acd-guest-3,-29.0930,-0.6493,-48.7146,426.2925,16.8092,443.1017,-429.3068,11.3698,-410.1856,-95.2452,-66.0578,-163.6184,5.1352,2.0628,5.3270,4.606
3,3,cd-set1--acd-guest-4,-29.6231,-0.0159,-43.0228,420.0687,-30.3678,389.7009,-416.5215,11.2809,-405.0136,-97.4461,-8.8723,-102.2474,5.0849,1.3330,4.9451,2.130
4,4,cd-set1--acd-guest-5,-28.8169,-0.9041,-46.8309,425.7315,-10.9172,414.8143,-435.9689,5.6626,-431.7111,-91.2312,-7.8776,-93.5205,5.0881,1.5514,5.0142,2.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,67,sampl6-CB8-8,-49.0064,3.3647,-70.6721,-2832.4453,-22.8929,-2855.3382,1726.3218,45.4060,1699.7389,-175.2083,-60.6497,-160.1198,6.2084,2.1236,5.6790,13.500
68,68,sampl6-CB8-9,-49.0064,-0.6967,-65.7360,-2832.4453,-50.2550,-2888.7892,1726.3218,57.0318,1711.3958,-175.5746,-59.1104,-160.3890,6.2230,1.6581,5.7092,8.680
69,69,sampl6-CB8-10,-49.0064,-2.4011,-75.9690,-2832.4453,-146.1591,-2978.6044,1726.3218,125.2640,1777.4060,-175.4034,-65.4112,-161.5333,6.2481,1.7718,5.6932,8.220
70,70,sampl6-CB8-11,-49.0064,-1.0304,-70.6514,-2832.4453,-45.4193,-2877.8646,1726.3218,157.8995,1743.1452,-175.3479,-200.3369,-224.9155,6.2942,1.6177,5.9772,7.770


In [6]:

# This is to create 4- fold for featurized dataset. includes physical features. X and y will be shuffled.
from models.dcFeaturizer import atom_features as get_atom_features


print('Data collected')

info = []
for pdb in list(PDBs.keys()):
    info.append(df[df['Ids'] == pdb][['pb_host_VDWAALS', 'pb_guest_VDWAALS', 'pb_complex_VDWAALS', 'gb_host_1-4EEL', 'gb_guest_1-4EEL', 'gb_Complex_1-4EEL',
       'gb_host_EELEC', 'gb_guest_EELEC', 'gb_Complex_EELEC', 'gb_host_EGB', 'gb_guest_EGB', 'gb_Complex_EGB', 'gb_host_ESURF', 'gb_guest_ESURF', 'gb_Complex_ESURF']].to_numpy()[0])



def featurize(molecule, info):
    atom_features = []
    for atom in molecule.GetAtoms():
        new_feature = get_atom_features(atom).tolist()
        position = molecule.GetConformer().GetAtomPosition(atom.GetIdx())
        new_feature += [atom.GetMass(), atom.GetAtomicNum(),atom.GetFormalCharge()]
        new_feature += [position.x, position.y, position.z]
        for neighbor in atom.GetNeighbors()[:2]:
            neighbor_idx = neighbor.GetIdx()
            new_feature += [neighbor_idx]
        for i in range(2 - len(atom.GetNeighbors())):
            new_feature += [-1]
        atom_features.append(np.concatenate([new_feature, info], 0))
    return np.array(atom_features)

X = []
y = []
for i, pdb in enumerate(list(PDBs.keys())):
    X.append(featurize(PDBs[pdb], info[i]))
    y.append(df[df['Ids'] == pdb]['Ex _G_(kcal/mol)'].to_numpy()[0])




No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
2025-05-01 23:49:17.166095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-01 23:49:17.254784: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical 

Data collected


In [8]:
print('Input data prepared')
X, y = shuffle(X, y)
K = 4
fold_size = len(X) // K
X_folds = [X[i*fold_size:(i + 1)*fold_size] for i in range(K)]
y_folds = [y[i*fold_size:(i + 1)*fold_size] for i in range(K)]

Input data prepared


In [9]:
dbfile = open('Datasets/X_folds_BFE_4.pkl', 'wb')
pickle.dump(X_folds, dbfile)                     
dbfile.close()

In [10]:
dbfile = open('Datasets/y_folds_BFE_4.pkl', 'wb')
pickle.dump(y_folds, dbfile)                     
dbfile.close()

In [3]:
import pickle
x_f = pickle.load(open('Datasets/X_folds_BFE_4.pkl', 'rb'))

In [23]:
x_f[0][0].shape,y[0]

((146, 41), -20.8002)